# BSF and VBC CellRanger import 

In [ ]:
library_load <- suppressMessages(
    
    suppressWarnings(
    
        list(
            
            # Seurat 
            library(Seurat),

            # SingleR
            library(SingleR), 
            library(SingleCellExperiment), 
            
            # Data 
            library(tidyverse),
            
            # Plot 
            library(ggplot2), 
            
            # Pyhton compatibility
            library(reticulate)
            
        )
    )
)

In [ ]:
# Configure reticulate 
use_condaenv(condaenv='p.3.10.16-FD20200109SPLENO', conda="/nobackup/peer/fdeckert/miniconda3/bin/conda", required=NULL)
py_config()

In [ ]:
options(warn=-1)

In [ ]:
random_seed <- 42
set.seed(random_seed)

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [ ]:
# Source files
source("bin/so_pl.R")
source("plotting_global.R")

# Import Seurat object

In [ ]:
so <- readRDS("data/scRNAseq/object/pp.rds")

# Annotation with bulkRNAseq experiment

In [ ]:
# Load reference data 
meta_ref <- readRDS("data/bulkRNAseq/object/meta.rds")
mat_ref <- readRDS("data/bulkRNAseq/object/cnt.rds") 

# Subset counts to samples in meta 
mat_ref <- mat_ref[, rownames(meta_ref)]

# Reference SingleCellExperiment
sce_ref <- SingleCellExperiment::SingleCellExperiment(list(counts=mat_ref), colData=meta_ref)
sce_ref <- scuttle::logNormCounts(sce_ref)

# Query SingleCellExperiment
sce_qry <- SingleCellExperiment::SingleCellExperiment(assays=list(counts=Seurat::GetAssayData(so, assay="RNA", layer="counts")))
sce_qry <- scuttle::logNormCounts(sce_qry)

# Predict labels
res <- SingleR::SingleR(test=sce_qry, ref=sce_ref, labels=sce_ref$celltype, assay.type.test="logcounts", assay.type.ref="logcounts", de.method="classic", fine.tune=FALSE) 
res_1 <- as.data.frame(res) %>% dplyr::select(pruned.labels, delta.next) %>% dplyr::rename(label_main_bulkrnaseq=pruned.labels, delta_score_main_bulkrnaseq=delta.next)
res_2 <- as.data.frame(res$scores) %>% dplyr::rename(cmo_score=cMo, prerpm_score=PreRPM, rpm_score=RPM, eb_score=EB)
label_main_bulkrnaseq <- cbind(res_1, res_2)

## Add cell type annotation to meta data

In [ ]:
so <- AddMetaData(so, label_main_bulkrnaseq)

# Plot

In [ ]:
options(repr.plot.width=4*7.5, repr.plot.height=7.5)

fplot_1 <- fplot(so, features="cmo_score", min_cutoff=0) + ggtitle("cMo score")
fplot_2 <- fplot(so, features="prerpm_score", min_cutoff=0) + ggtitle("PreRPM score")
fplot_3 <- fplot(so, features="rpm_score", min_cutoff=0) + ggtitle("RPM score")
fplot_4 <- fplot(so, features="eb_score", min_cutoff=0) + ggtitle("EB score")

fplot_1 + fplot_2 + fplot_3 + fplot_4 + patchwork::plot_layout(ncol=4)

# Save results 

In [ ]:
write.csv(label_main_bulkrnaseq, "result/singler/bulkRNAseq/singler_res.csv")

# Session info 

In [ ]:
sessionInfo()